#  Transforming Collections Data to Linked Art 
## National Gallery of Art

## Import What We Need for Notebook
See the [Transform Introduction](01-00-Transform-Intro.ipynb) for a description of the Python libraries used

In [2]:
try:
    import json
except:
    !pip install json
    import json 
    
try:
    import requests
except:
    !pip install requests
    import requests
    
try:
    import cromulent
except:
    !pip install cromulent
    import cromulent
    
import csv

from cromulent.model import factory
    
from lib import linkedart as la


## Input Data
The input data file is from https://github.com/NationalGalleryOfArt/opendata/tree/main/data

In [3]:
file = './data/nga/input/objects.csv'

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

for obj in allObjects:
    print(json.dumps(obj,indent=2))
    break

{
  "objectid": "113260",
  "accessioned": "1",
  "accessionnum": "2000.127.20.1-193",
  "locationid": "",
  "title": "Lithographs, Volume 9",
  "displaydate": "",
  "beginyear": "1804",
  "endyear": "1866",
  "visualbrowsertimespan": "1801 to 1825",
  "medium": "book of lithographs",
  "dimensions": "",
  "inscription": "",
  "markings": "",
  "attributioninverted": "Gavarni, Paul",
  "attribution": "Paul Gavarni",
  "creditline": "Ailsa Mellon Bruce Fund",
  "classification": "Volume",
  "subclassification": "",
  "visualbrowserclassification": "volume",
  "parentid": "",
  "isvirtual": "0",
  "departmentabbr": "CG-E",
  "portfolio": "",
  "series": "",
  "volume": "",
  "watermarks": "",
  "lastdetectedmodification": "2020-05-06 22:01:32.06-04",
  "customprinturl": ""
}


## Map fields

In [4]:
  
 mapp =  {
    "id":"objectid",
    "accession_number":"accessionnum",
    "accession_date": "",
    "classification" : "classification",
    "title": "title",
    "alt_title": "",
    "notes": "",
    "date_created":"displaydate",
    "date_created_earliest": "beginyear",
    "date_created_latest": "endyear",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"attribution",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "dimensions",
    "created_provenance": "" ,
    "credit_line": "creditline",
    "collection" : "departmentabbr",
    "current_status" : "",
    "current_owner" : ""
}

In [6]:
def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = {"name": obj[key]}
                else:
                    objProp[prop] = obj[key]
    objProp["current_owner"] = {"name":"National Gallery of Art",
                                "location":"Washington, D.C., United States",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    return objProp   

In [8]:
#  baseURI for JSON-LD document
baseURI = "https://www.nga.gov/collection/"

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp)

    id = objProp["id"]
    object_uri = baseURI + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
    # write to file 
    text_file = open("./data/nga/output/json/all/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()